# ResNet50

## 1. Imports:

In [2]:
# Importing necessary libraries
# For data manipulation and numerical operations
import pandas as pd
import numpy as np
from numpy import save, load, asarray

# Deep learning and neural network building
import keras
from keras.layers import Dense, Convolution1D, Flatten, Convolution2D
from tensorflow.keras.layers import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l1
import tensorflow as tf
from keras.utils import to_categorical, plot_model
from keras.models import Sequential, Model
from keras.layers import Input, Dropout, Activation, Add
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPool1D, ZeroPadding1D, LSTM, Bidirectional

# Visualization tools
import matplotlib.pyplot as plt
import seaborn as sns

# Scientific computing and file loading
from scipy import optimize
from scipy.io import loadmat

# For merging layers in TensorFlow
from tensorflow.keras.layers import concatenate

## 2. Data Importing:

In [ ]:
# Load training and testing datasets (features and labels)
X_train = load(r"3. Data set generation\ECG_training.npy", allow_pickle=True)
y_train = load(r"3. Data set generation\ECG_training_labels.npy", allow_pickle=True)
X_test  = load(r"3. Data set generation\ECG_testing.npy", allow_pickle=True)
y_test  = load(r"3. Data set generation\ECG_testing_labels.npy", allow_pickle=True)

## 3. ResNet50 Structure:

In [3]:
# Define the identity block of ResNet50
def identity_block(X, f, filters):
    # Filters for each convolution
    F1, F2, F3 = filters
    
    # Save input for shortcut connection
    X_shortcut = X
    
    # Convolution layers
    X = Conv1D(filters=64, kernel_size=5, activation='relu', strides=1, padding='same')(X)
    X = BatchNormalization()(X)

    # Additional convolutions and activations
    X = Conv1D(filters=64, kernel_size=5, activation='relu', strides=1, padding='same')(X)
    X = BatchNormalization()(X)

    # Final set of convolutions
    X = Conv1D(filters=F3, kernel_size=1, activation='relu', strides=1, padding='same')(X)
    X = BatchNormalization()(X)

    # Shortcut connection
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

# Define convolutional block
def convolutional_block(X, f, filters, s=2):
    F1, F2, F3 = filters
    
    # Save shortcut
    X_shortcut = X

    # Main path with strides to downsample
    X = Conv1D(F1, 1, activation='relu', strides=s)(X)
    X = BatchNormalization()(X)
    
    # Other convolutions
    X = Conv1D(F2, f, activation='relu', strides=1, padding='same')(X)
    X = BatchNormalization()(X)

    # Shortcut with same dimensions
    X_shortcut = Conv1D(F3, 1, strides=s)(X_shortcut)
    X_shortcut = BatchNormalization()(X_shortcut)

    # Merge and activate
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

# ResNet50 architecture
def ResNet50(input_shape):
    X_input = Input(input_shape)

    # Initial layer
    X = ZeroPadding1D(3)(X_input)
    X = Conv1D(64, 7, strides=2)(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)

    # Add blocks for feature extraction
    X = convolutional_block(X, 3, [64, 64, 256], s=1)
    X = identity_block(X, 3, [64, 64, 256])
    
    X = GlobalAveragePooling1D()(X)
    X = Dense(2, activation='sigmoid')(X)

    return Model(inputs=X_input, outputs=X, name='ResNet50')